# CNN
## Imports

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

#import numpy as np
#import matplotlib.pyplot as plt

## Network

In [6]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        #The words kernal and filter are interchangeable
        self.conv1=nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
        self.conv2=nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1=nn.Linear(in_features= 12*4*4, out_features=120)
        self.fc2=nn.Linear(in_features= 120, out_features=60)
        self.out=nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t


## Data

In [7]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

## Training


In [10]:
def train_model(device, epochs):
    network = Network()
    print(f"training on: {device}")


    def get_num_correct(preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
    optimizer = optim.Adam(network.parameters(),lr=0.01)

    for epoch in range(epochs):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            images, labels = batch[0].to(device), batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels)

            optimizer.zero_grad()
            loss.backward() # calculates gradient
            optimizer.step() # update weights

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)

        print(f"epoch: {epoch+1}/{epochs}, total_correc: {total_correct}, loss: {total_loss}")

### on cpu

cpu mac: Wall time: 3min 10s
cpu home: 

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_model(device, 2)


# cpu home: 
# gpu home:


training on: cpu
epoch: 0/2, total_correc: 45603, loss: 373.7340459227562
epoch: 1/2, total_correc: 50966, loss: 245.57134871184826


###  on gpu

gpu K80: 1 min 36 sec

In [12]:
device = torch.device("cuda:0")
train_model(device, 2)

## Prediction

In [35]:
sample = next(iter(train_set))
image, label = sample

output = network(image.unsqueeze(0))
pred = torch.argmax(output, dim=1).item()

print(f"Our pred: {pred} actual label: {label}")

Our pred: 5 actual label: 9
